# Ver2.0

## 01. cnn

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
import numpy as np

# -----------------------------
# 1. 데이터 준비
# -----------------------------
train_dir = "/content/drive/MyDrive/brain_ct/train"
test_dir = "/content/drive/MyDrive/brain_ct/test"

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# -----------------------------
# 2. CNN 모델 정의
# -----------------------------
def create_simple_cnn(input_shape=(224,224,3)):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

cnn_model = create_simple_cnn()
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.summary()

# -----------------------------
# 3. 학습
# -----------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

cnn_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    callbacks=[early_stop]
)

# -----------------------------
# 4. 모델 저장
# -----------------------------
cnn_model.save("/content/drive/MyDrive/cnn_brain_ct.h5")

# -----------------------------
# 5. 테스트 데이터 정확도
# -----------------------------
loss, acc = cnn_model.evaluate(test_generator)
print(f"CNN 모델 테스트 정확도: {acc*100:.2f}%")

# -----------------------------
# 6. 임의 이미지 예측
# -----------------------------
img_path = "/content/drive/MyDrive/brain_ct/test/hemorrhage/ds1_0_hemorrhage_1026_IMG-0001-00073.jpg"
img = image.load_img(img_path, target_size=img_size)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

pred = cnn_model.predict(x)[0][0]
label = "hemorrhage" if pred >= 0.5 else "normal"
print(f"CNN 모델 예측: {label} ({pred:.4f})")


Found 5523 images belonging to 2 classes.
Found 1381 images belonging to 2 classes.


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 95s 529ms/step - accuracy: 0.6538 - loss: 0.7213 - val_accuracy: 0.7741 - val_loss: 0.5768
Epoch 2/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 88s 505ms/step - accuracy: 0.7523 - loss: 0.4961 - val_accuracy: 0.8516 - val_loss: 0.3502
Epoch 3/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 87s 505ms/step - accuracy: 0.8408 - loss: 0.3733 - val_accuracy: 0.9095 - val_loss: 0.2040
Epoch 4/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 87s 504ms/step - accuracy: 0.9016 - loss: 0.2401 - val_accuracy: 0.9290 - val_loss: 0.1553
Epoch 5/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 86s 499ms/step - accuracy: 0.9333 - loss: 0.1659 - val_accuracy: 0.9529 - val_loss: 0.1233
Epoch 6/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 88s 508ms/step - accuracy: 0.9472 - loss: 0.1300 - val_accuracy: 0.9776 - val_loss: 0.0673
Epoch 7/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 88s 509ms/step - accuracy: 0.9600 - loss: 0.1059 - val_accuracy: 0.9884 - val_loss: 0.0470
Epoch 8/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 87s 504ms/step - accuracy: 0.9685 - loss: 0

44/44 ━━━━━━━━━━━━━━━━━━━━ 9s 207ms/step - accuracy: 0.9868 - loss: 0.0565
CNN 모델 테스트 정확도: 99.13%
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 798ms/step
CNN 모델 예측: normal (0.0001)


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
import numpy as np
from google.colab import drive


drive.mount('/content/drive')

import os
print(os.listdir("/content/drive/MyDrive"))

print(os.listdir("/content/drive/MyDrive/brain_ct"))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['Colab Notebooks', 'How to get started with Drive.pdf', 'Gdriveplayer', 'Fantastic Beasts: The Crimes of Grindelwald, 2018.mp4', 'Longmire.S2E6.mp4', 'The.Marine.6.Close.Quarters.2018.BluRay.720p.FMZM.mkv', 'Screenshot_20190124-002601 (1).png', 'Screenshot_20190124-002535 (1).png', 'Screenshot_20180323-140349.png', 'Screenshot_20181018-142226 (1).png', 'Screenshot_20190124-002535.png', 'Screenshot_20190124-002601.png', 'Screenshot_20180606-040330 (1).png', 'Screenshot_20181018-142029.png', 'Screenshot_20180420-141245.png', 'Screenshot_20180924-125819.png', 'Screenshot_20181018-142216.png', 'Screenshot_20180324-222653.png', 'Screenshot_20180523-023838.png', 'Screenshot_20181018-142251.png', 'Screenshot_20180307-220521.png', 'Screenshot_20180324-222705.png', 'Screenshot_20180310-011218.png', 'Screenshot_20170926-010426.png', 'Screenshot_20180606-041916.png', '

In [9]:
# 필수 라이브러리 설치 (필요 시)
!pip install tensorflow opencv-python matplotlib numpy pandas scikit-learn tqdm

# 기본 라이브러리 임포트
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# 구글 드라이브 경로 설정
base_path = "/content/drive/MyDrive"

# 작업 폴더를 하나 지정 (원하는 폴더명으로 바꿔도 됨)
project_path = os.path.join(base_path, "Colab Notebooks")
print("작업 경로:", project_path)

# 폴더 목록 확인 (정상 접근 확인)
print(os.listdir(project_path))


작업 경로: /content/drive/MyDrive/Colab Notebooks
['brain_ct.ipynb']


# 최적의 학습률을 찾는 하이퍼파라미터 작업
0.01, 0.001, 0.0001 로 하이퍼파라미터 테스트로 최적의 학습률 찾기 -> 정확도

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# 예시 데이터 (여기서는 랜덤 데이터로 대체)
# 실제로는 CT 이미지와 라벨로 대체해야 함
import numpy as np
num_samples = 100
img_height, img_width = 64, 64
num_classes = 2

X_train = np.random.rand(num_samples, img_height, img_width, 1).astype(np.float32)
y_train = tf.keras.utils.to_categorical(np.random.randint(0, num_classes, num_samples), num_classes)

# 테스트할 하이퍼파라미터
learning_rates = [0.01, 0.001, 0.0001]
epochs_list = [5]  # 각 에폭은 5로 고정

# 결과 저장용
results = {}

for lr in learning_rates:
    for epochs in epochs_list:
        # 모델 정의
        model = Sequential([
            Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width,1)),
            MaxPooling2D((2,2)),
            Flatten(),
            Dense(64, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])

        # 옵티마이저 설정
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        # 학습
        print(f"Training with learning rate={lr}, epochs={epochs}")
        history = model.fit(X_train, y_train, epochs=epochs, batch_size=8, verbose=1)

        # 최종 정확도 저장
        results[f"lr_{lr}_epochs_{epochs}"] = history.history['accuracy'][-1]

print("모든 테스트 완료!")
print(results)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training with learning rate=0.01, epochs=5
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - accuracy: 0.5405 - loss: 5.1141
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5750 - loss: 0.6795
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6584 - loss: 0.6311 
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5827 - loss: 0.6610 
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7261 - loss: 0.5360 
Training with learning rate=0.001, epochs=5
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.4311 - loss: 1.6031
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6061 - loss: 0.6992 
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5378 - loss: 0.6991 
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6595 - loss: 0.6514 
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5829 - loss: 0.6598 
Training with learning rate=0.0001, epochs=5
Epoch 1/5
13/13 ━━━━━━━━━━━━━

In [1]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# -----------------------------
# 학습률 후보
# -----------------------------
learning_rates = [0.01, 0.001, 0.0001]

results = {}

# -----------------------------
# 학습률별 실험
# -----------------------------
for lr in learning_rates:
    print("\n====================================")
    print(f"📌 Learning Rate = {lr} 실험 시작")
    print("====================================")

    # 모델 새로 생성
    model = create_simple_cnn()
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # 조기 종료
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=2,
        restore_best_weights=True
    )

    # 모델 학습 (epochs = 5)
    history = model.fit(
        train_generator,
        validation_data=test_generator,
        epochs=5,
        callbacks=[early_stop],
        verbose=1
    )

    # 검증 정확도 저장
    val_acc = max(history.history['val_accuracy'])
    results[lr] = val_acc
    print(f"➡️ LR {lr} 최종 검증 정확도: {val_acc:.4f}")

# -----------------------------
# 결과 비교 출력
# -----------------------------
print("\n===== 학습률별 성능 요약 =====")
for lr, acc in results.items():
    print(f"LR={lr} → Val Accuracy = {acc:.4f}")

# 가장 좋은 학습률 찾기
best_lr = max(results, key=results.get)
print(f"\n🎯 가장 성능이 좋은 학습률: {best_lr}")


📌 Learning Rate = 0.01 실험 시작


NameError: name 'create_simple_cnn' is not defined

## 2. ResNet from scratch

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import numpy as np

# -----------------------------
# 1. 데이터 준비
# -----------------------------
train_dir = "/content/drive/MyDrive/brain_ct/train"
test_dir = "/content/drive/MyDrive/brain_ct/test"

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# -----------------------------
# 2. ResNet from scratch 정의
# -----------------------------
def create_resnet_from_scratch(input_shape=(224,224,3)):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (7,7), strides=2, padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    for filters in [64,128,256]:
        x = layers.Conv2D(filters, (3,3), activation='relu', padding='same')(x)
        x = layers.Conv2D(filters, (3,3), activation='relu', padding='same')(x)
        x = layers.MaxPooling2D(2,2)(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs, outputs)
    return model

resnet_scratch = create_resnet_from_scratch()
resnet_scratch.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
resnet_scratch.summary()

# -----------------------------
# 3. 학습
# -----------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

resnet_scratch.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    callbacks=[early_stop]
)

# -----------------------------
# 4. 모델 저장
# -----------------------------
resnet_scratch.save("/content/drive/MyDrive/resnet_scratch_brain_ct.h5")

# -----------------------------
# 5. 테스트 데이터 정확도
# -----------------------------
loss, acc = resnet_scratch.evaluate(test_generator)
print(f"ResNet scratch 테스트 정확도: {acc*100:.2f}%")

# -----------------------------
# 6. 임의 이미지 예측
# -----------------------------
img_path = "/content/drive/MyDrive/brain_ct/test/hemorrhage/ds1_0_hemorrhage_1026_IMG-0001-00073.jpg"
img = image.load_img(img_path, target_size=img_size)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

pred = resnet_scratch.predict(x)[0][0]
label = "hemorrhage" if pred >= 0.5 else "normal"
print(f"ResNet scratch 예측: {label} ({pred:.4f})")


Found 5523 images belonging to 2 classes.
Found 1381 images belonging to 2 classes.


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 112, 112, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 14, 14, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,223,041 (4.67 MB)

 Trainable params: 1,223,041 (4.67 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 103s 548ms/step - accuracy: 0.6178 - loss: 0.6839 - val_accuracy: 0.6104 - val_loss: 0.6683
Epoch 2/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 86s 499ms/step - accuracy: 0.6019 - loss: 0.6719 - val_accuracy: 0.6104 - val_loss: 0.6773
Epoch 3/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 88s 506ms/step - accuracy: 0.6264 - loss: 0.6583 - val_accuracy: 0.6524 - val_loss: 0.6379
Epoch 4/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 87s 504ms/step - accuracy: 0.6464 - loss: 0.6396 - val_accuracy: 0.6524 - val_loss: 0.6805
Epoch 5/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 87s 505ms/step - accuracy: 0.6655 - loss: 0.6212 - val_accuracy: 0.6988 - val_loss: 0.5636
Epoch 6/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 86s 499ms/step - accuracy: 0.6997 - loss: 0.5659 - val_accuracy: 0.7038 - val_loss: 0.5433
Epoch 7/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 89s 514ms/step - accuracy: 0.7221 - loss: 0.5293 - val_accuracy: 0.8045 - val_loss: 0.4445
Epoch 8/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 88s 507ms/step - accuracy: 0.7795 - loss: 

44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 146ms/step - accuracy: 0.7090 - loss: 0.5729
ResNet scratch 테스트 정확도: 84.50%
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 864ms/step
ResNet scratch 예측: normal (0.0000)


# ResNet50 전이학습

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
import numpy as np

# -----------------------------
# 1. 데이터 준비
# -----------------------------
train_dir = "/content/drive/MyDrive/brain_ct/train"
test_dir = "/content/drive/MyDrive/brain_ct/test"

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# -----------------------------
# 2. ResNet50 전이학습 정의 (freeze)
# -----------------------------
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in base_model.layers:
    layer.trainable = False

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

resnet_transfer = Model(inputs=base_model.input, outputs=outputs)
resnet_transfer.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
resnet_transfer.summary()

# -----------------------------
# 3. 학습
# -----------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

resnet_transfer.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    callbacks=[early_stop]
)

# -----------------------------
# 4. 모델 저장
# -----------------------------
resnet_transfer.save("/content/drive/MyDrive/resnet_transfer_brain_ct.h5")

# -----------------------------
# 5. 테스트 데이터 정확도
# -----------------------------
loss, acc = resnet_transfer.evaluate(test_generator)
print(f"ResNet50 transfer 테스트 정확도: {acc*100:.2f}%")

# -----------------------------
# 6. 임의 이미지 예측
# -----------------------------
img_path = "/content/drive/MyDrive/brain_ct/test/hemorrhage/ds1_0_hemorrhage_1026_IMG-0001-00073.jpg"
img = image.load_img(img_path, target_size=img_size)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

pred = resnet_transfer.predict(x)[0][0]
label = "hemorrhage" if pred >= 0.5 else "normal"
print(f"ResNet50 transfer 예측: {label} ({pred:.4f})")


Found 5523 images belonging to 2 classes.
Found 1381 images belonging to 2 classes.


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_8[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,850,113 (90.98 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 112s 585ms/step - accuracy: 0.5836 - loss: 0.6764 - val_accuracy: 0.6379 - val_loss: 0.6057
Epoch 2/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 126s 524ms/step - accuracy: 0.6463 - loss: 0.6248 - val_accuracy: 0.7299 - val_loss: 0.5491
Epoch 3/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 141s 520ms/step - accuracy: 0.6953 - loss: 0.5849 - val_accuracy: 0.7393 - val_loss: 0.5150
Epoch 4/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 91s 524ms/step - accuracy: 0.7087 - loss: 0.5575 - val_accuracy: 0.7603 - val_loss: 0.4893
Epoch 5/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 91s 525ms/step - accuracy: 0.7209 - loss: 0.5469 - val_accuracy: 0.7668 - val_loss: 0.4729
Epoch 6/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 90s 522ms/step - accuracy: 0.7406 - loss: 0.5157 - val_accuracy: 0.7495 - val_loss: 0.4751
Epoch 7/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 90s 520ms/step - accuracy: 0.7555 - loss: 0.4883 - val_accuracy: 0.7610 - val_loss: 0.4766
Epoch 8/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 90s 517ms/step - accuracy: 0.7533 - loss

44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 157ms/step - accuracy: 0.7336 - loss: 0.4692
ResNet50 transfer 테스트 정확도: 80.45%
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
ResNet50 transfer 예측: hemorrhage (0.5980)


1. 데이터셋 규모와 모델 복잡도

CNN 모델: 비교적 간단한 구조, 파라미터 수 적음 → 5000장 정도의 데이터셋에서 충분히 학습 가능

ResNet from scratch: 깊은 모델, 파라미터 많음 → 데이터가 적으면 오히려 과적합(overfitting)되거나 제대로 학습되지 않을 수 있음

ResNet50 transfer: 사전 학습된 가중치를 freeze → Dense만 학습

만약 데이터가 ResNet50이 ImageNet으로 학습된 특성과 많이 다르면, Dense만 학습하는 것이 최적화가 충분하지 않을 수 있음

2. 학습 조건

에포크 수가 적으면 간단한 CNN은 빨리 수렴

ResNet scratch는 깊어서 충분히 학습되지 않았을 가능성

ResNet50 transfer는 Dense만 학습 → 일부 데이터에서는 CNN보다 예측력이 낮을 수 있음

3. 결론

작은/중간 규모 데이터셋에서는 단순 CNN이 더 잘 맞는 경우가 많습니다

ResNet 계열은 데이터가 많거나 사전학습 가중치 fine-tuning을 할 때 더 강력해집니다

# ResNet50 transfer 모델을 fine-tuning

1. 클래스 균형 맞추기
ImageDataGenerator에서 class_weight를 설정하여 hemorrhage/normal 샘플 비율을 맞춤

2. 현재 Dense만 학습했기 때문에 hemorrhage 특징은 잘 잡지만 normal 특징은 부족하기에 해결하려면 부분 unfreeze + fine-tuning + 클래스 균형 + augmentation 조합

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

# -----------------------------
# 1. 데이터 준비
# -----------------------------
train_dir = "/content/drive/MyDrive/brain_ct/train"
test_dir = "/content/drive/MyDrive/brain_ct/test"

# 데이터 증강 (preprocess_input 포함)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128,128),  # 이미지 크기 축소
    batch_size=32,           # 배치 사이즈 증가
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# -----------------------------
# 2. ResNet50 base model 불러오기 (pretrained)
# -----------------------------
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(128,128,3)
)

# -----------------------------
# 3. Top layers 구성
# -----------------------------
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# -----------------------------
# 4. 학습: base model freeze (Dense만 학습)
# -----------------------------
for layer in base_model.layers:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=5,                 # epoch 최소화
    validation_data=test_generator
)

# -----------------------------
# 5. 모델 저장
# -----------------------------
model.save("/content/drive/MyDrive/resnet_transfer_fast_brain_ct.h5")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 5523 images belonging to 2 classes.
Found 1381 images belonging to 2 classes.
Epoch 1/5
173/173 ━━━━━━━━━━━━━━━━━━━━ 820s 5s/step - accuracy: 0.6845 - loss: 0.7128 - val_accuracy: 0.8074 - val_loss: 0.4306
Epoch 2/5
173/173 ━━━━━━━━━━━━━━━━━━━━ 461s 3s/step - accuracy: 0.8027 - loss: 0.4268 - val_accuracy: 0.8472 - val_loss: 0.3804
Epoch 3/5
173/173 ━━━━━━━━━━━━━━━━━━━━ 468s 3s/step - accuracy: 0.8377 - loss: 0.3517 - val_accuracy: 0.8617 - val_loss: 0.3449
Epoch 4/5
173/173 ━━━━━━━━━━━━━━━━━━━━ 494s 3s/step - accuracy: 0.8532 - loss: 0.3329 - val_accuracy: 0.8573 - val_loss: 0.3313
Epoch 5/5
173/173 ━━━━━━━━━━━━━━━━━━━━ 464s 3s/step - accuracy: 0.8662 - loss: 0.3132 - val_accuracy: 0.8783 - val_loss: 0.2963


# streamlit

In [ ]:
# streamlit_brain_ct_app_sidebar_single_model.py
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

st.title("뇌출혈 예측 웹앱")
st.write("사이드바에서 이미지를 업로드하고 모델을 선택하세요.")

# -----------------------------
# 1. 사이드바: 이미지 업로드
# -----------------------------
uploaded_file = st.sidebar.file_uploader("이미지 업로드", type=["jpg","png","jpeg"])

# -----------------------------
# 2. 사이드바: 모델 선택
# -----------------------------
model_option = st.sidebar.radio(
    "모델 선택",
    ("CNN", "ResNet from scratch", "ResNet50 transfer", "ResNet50 fine-tuned")
)

# -----------------------------
# 3. 모델 로드 (캐시)
# -----------------------------
@st.cache_resource
def load_models():
    cnn_model = load_model("model/cnn_brain_ct.h5")
    resnet_scratch = load_model("model/resnet_scratch_brain_ct.h5")
    resnet_transfer = load_model("model/resnet_transfer_brain_ct.h5")
    resnet_finetuned = load_model("model/resnet_transfer_fast_brain_ct.h5")  # Dense-only 빠른 학습
    return cnn_model, resnet_scratch, resnet_transfer, resnet_finetuned

cnn_model, resnet_scratch, resnet_transfer, resnet_finetuned = load_models()

# -----------------------------
# 4. 이미지 전처리 및 예측
# -----------------------------
if uploaded_file is not None:
    img = Image.open(uploaded_file).convert('RGB')
    st.image(img, caption='업로드된 이미지', use_container_width=True)

    # 모델별 입력 크기 설정
    def get_img_array(model_name, img):
        if model_name == "ResNet50 fine-tuned":
            size = (128,128)
        else:
            size = (224,224)
        img_array = image.img_to_array(img.resize(size))
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        return img_array

    # 예측 함수
    def predict_label(model, img_array):
        pred = model.predict(img_array)[0][0]
        label = "hemorrhage" if pred >= 0.5 else "normal"
        return label, pred

    # 모델 선택
    if model_option == "CNN":
        model = cnn_model
    elif model_option == "ResNet from scratch":
        model = resnet_scratch
    elif model_option == "ResNet50 transfer":
        model = resnet_transfer
    else:  # fine-tuned
        model = resnet_finetuned

    img_array = get_img_array(model_option, img)
    label, pred = predict_label(model, img_array)
    st.write(f"### 선택 모델: {model_option}")
    st.write(f"결과: **{label}**")
    st.write(f"Hemorrhage 확률: {pred:.4f}")
